In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from saex.trainer_cache import main
main()

Creating SAE...
Loading checkpoint (weights/bloom-gpt2s-1.safetensors)...
Loading model...


Some weights of the model checkpoint at gpt2 were not used when initializing FlaxGPT2Model: {('h', '9', 'attn', 'bias'), ('h', '8', 'attn', 'bias'), ('h', '10', 'attn', 'bias'), ('h', '5', 'attn', 'bias'), ('h', '7', 'attn', 'bias'), ('h', '4', 'attn', 'bias'), ('h', '0', 'attn', 'bias'), ('h', '11', 'attn', 'bias'), ('h', '1', 'attn', 'bias'), ('h', '6', 'attn', 'bias'), ('h', '2', 'attn', 'bias'), ('h', '3', 'attn', 'bias')}
- This IS expected if you are initializing FlaxGPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxGPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading dataset...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Training for 10000 iterations


Training SAE:   0%|          | 0/10000 [00:21<?, ?it/s]/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/jax/_src/interpreters/mlir.py:920: UserWarning: Some donated buffers were not usable: ShapedArray(float16[1024,768]).
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn("Some donated buffers were not usable:"
Training SAE:   5%|▌         | 512/10000 [00:56<17:18,  9.13it/s, dead=0.1739502, l0=16.057617, loss=0.14796296, loss_reconstruction=0.081810966, loss_sparsity=41.344994, max_time_since_fired=512.0, var_explained=0.9207162]  


KeyboardInterrupt: 